# **Bayesian Updating with Conjugate Priors**
---
---

### **Conjugate Prior:** posterior with same distribution as the prior
- certain likelihood functions with specific prior distributions -> posterior having the same distribution as the prior



> 
> ### P(θ) such that P(θ|D) = P(θ)
> - chosen *Prior Distribution* for the *Likelihood Function* = a *Posterior Distribution* the same as the *Prior Distribution* 
> - Allows you to skip `posterior = likelihood * prior` computation (if you know it is a conjugate prior)
> - implied **Closed-Form Expression** for the *prior disribution* (you already know what the maximum posterior will be)
>      - *Closed-Form Expression* = finite number of standard operations

### Import Libraries

In [1]:
import numpy as np
import scipy.stats as stats

## Calculate Posterior of Binomial Likelihood (Conjugate Priors)
---
---
Medium Article: [Conjugate Prior](https://towardsdatascience.com/conjugate-prior-explained-75957dc80bfb)

- `Θ` = probability of success
- goal = pick the `Θ` that maximizes the postrior probability

### Generate Dataset and Parameters
---

In [16]:
# Generate Dataset
success_prob = 0.3
data = np.random.binomial(n=1, p=success_prob, size=1000)

# Θ
theta_range = np.linspace(0,1,1000)

# Prior P(Θ)
    # Beta Distribution: Probability Distribution on Probabilities
Alpha = 2
Beta = 8
theta_range_prior = theta_range + 0.0001
c_prior = stats.beta.cdf(x=theta_range_prior, a=Alpha, b=Beta) - stats.beta.cdf(x=theta_range, a=Alpha, b=Beta)
pd_prior = stats.beta.pdf(x=theta_range, a=Alpha, b=Beta)

In [17]:
collections.Counter(c_prior) == collections.Counter(pd_prior)

False